## Data Preparation
Let's load this data into Python.

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reading ratings file
# Ignore the timestamp column
ratings = pd.read_csv('ratings.csv', encoding='latin-1', usecols=['user_id', 'movie_id', 'rating'])

# Reading users file
users = pd.read_csv('users.csv', encoding='latin-1', usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])

# Reading movies file
movies = pd.read_csv('movies.csv', encoding='latin-1', usecols=['movie_id', 'title', 'genres'])

In [82]:
print(movies.shape)

(100, 3)


In [83]:
ratings.head(5)

,user_id,movie_id,rating
0,1,1,3
1,1,3,5
2,1,4,4
3,1,5,3
4,1,6,3


In [84]:
users.head(5)

,user_id,gender,zipcode,age_desc,occ_desc
0,1,F,48067,Under 18,K-12 student
1,2,M,70072,56+,self-employed
2,3,M,55117,25-34,scientist
3,4,M,2460,45-49,executive/managerial
4,5,M,55455,25-34,writer


In [85]:
movies.head(5)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## Content-Based Recommendation Model

Computes similarity between movies based on movie genres. It will suggest movies that are most similar to a particular movie based on its genre.

In [86]:
# Break up the big genre string into a string array
movies['genres'] = movies['genres'].apply(lambda x: str(x).split('|'))
print(movies.shape)
movies.head(5)

(100, 3)


,movie_id,title,genres
0,1,Toy Story (1995),"[Animation, Children's, Comedy]"
1,2,Jumanji (1995),"[Adventure, Children's, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [87]:
# Convert genres to string value
movies['genres'] = movies['genres'].fillna("").astype('str')
movies.head(5)

,movie_id,title,genres
0,1,Toy Story (1995),"['Animation', ""Children's"", 'Comedy']"
1,2,Jumanji (1995),"['Adventure', ""Children's"", 'Fantasy']"
2,3,Grumpier Old Men (1995),"['Comedy', 'Romance']"
3,4,Waiting to Exhale (1995),"['Comedy', 'Drama']"
4,5,Father of the Bride Part II (1995),['Comedy']


In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])
tfidf_matrix.shape

(100, 45)

In [89]:
tfidf_matrix

<100x45 sparse matrix of type '<class 'numpy.float64'>'
	with 246 stored elements in Compressed Sparse Row format>

In [90]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
#cosine_sim[:4, :4]
cosine_sim

array([[1.        , 0.15337409, 0.12551391, ..., 0.        , 0.        ,
        0.        ],
       [0.15337409, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.12551391, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.25861841],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.25861841, 0.        ,
        1.        ]])

In [91]:
# Build a 1-dimensional array with movie titles
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

#TODO: Function that get movie recommendations based on the cosine similarity score of movie genres
def genre_recommendations(title):
    id = np.where(titles==title)[0][0]
    rec = sorted(titles, key=lambda x: -cosine_sim[id][np.where(titles==x)[0][0]])
    rec = rec[1:]
    result = pd.DataFrame(columns=["ID", "Title"])
    while len(rec) > 0:
      top = rec[0]
      rec = rec[1:]
      result = result.append({"ID": np.where(titles==top)[0][0], "Title": top}, ignore_index=True)

    return result
genre_recommendations('Toy Story (1995)').head(20)

,ID,Title
0,12,Balto (1995)
1,86,Dunston Checks In (1996)
2,33,Babe (1995)
3,47,Pocahontas (1995)
4,4,Father of the Bride Part II (1995)
5,18,Ace Ventura: When Nature Calls (1995)
6,37,It Takes Two (1995)
7,51,Mighty Aphrodite (1995)
8,62,Don't Be a Menace to South Central While Drink...
9,64,Bio-Dome (1996)


Let's try and get the top recommendations for a few movies and see how good the recommendations are.

In [92]:
genre_recommendations('Toy Story (1995)').head(20)

,ID,Title
0,12,Balto (1995)
1,86,Dunston Checks In (1996)
2,33,Babe (1995)
3,47,Pocahontas (1995)
4,4,Father of the Bride Part II (1995)
5,18,Ace Ventura: When Nature Calls (1995)
6,37,It Takes Two (1995)
7,51,Mighty Aphrodite (1995)
8,62,Don't Be a Menace to South Central While Drink...
9,64,Bio-Dome (1996)


## Collaborative Filtering Recommendation Model


Use the file **ratings.csv** first as it contains User ID, Movie IDs and Ratings. These three elements are all needed for determining the similarity of the users based on their ratings for a particular movie.


In [93]:
# Fill NaN values in user_id and movie_id column with 0
ratings['user_id'] = ratings['user_id'].fillna(0)
ratings['movie_id'] = ratings['movie_id'].fillna(0)

# Replace NaN values in rating column with average of all values
ratings['rating'] = ratings['rating'].fillna(ratings['rating'].mean())
ratings

,user_id,movie_id,rating
0,1,1,3
1,1,3,5
2,1,4,4
3,1,5,3
4,1,6,3
...,...,...,...
7507,100,96,4
7508,100,97,4
7509,100,98,5
7510,100,99,5


Take a random sample of 20,000 ratings (2%) (due to limitation of personal laptop)

In [94]:
# Randomly sample 1% of the ratings dataset
small_data = ratings.sample(frac=0.02)
# Check the sample info
print(small_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 2765 to 3932
Data columns (total 3 columns):
user_id     150 non-null int64
movie_id    150 non-null int64
rating      150 non-null int64
dtypes: int64(3)
memory usage: 4.7 KB
None


In [0]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(small_data, test_size=0.2)

In [103]:
# Create two user-item matrices, one for training and another for testing
train_data_matrix = train_data.as_matrix(columns = ['user_id', 'movie_id', 'rating'])
test_data_matrix = test_data.as_matrix(columns = ['user_id', 'movie_id', 'rating'])

# Check their shape
print(train_data_matrix.shape)
print(test_data_matrix[:4, :4])

(120, 3)
[[ 2 80  3]
 [96 14  3]
 [62  9  4]
 [79 30  5]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


Now I use the **pairwise_distances** function from sklearn [Pearson Correlation Coefficient](https://stackoverflow.com/questions/1838806/euclidean-distance-vs-pearson-correlation-vs-cosine-similarity). This method provides a safe way to take a distance matrix as input, while preserving compatibility with many other algorithms that take a vector array.

In [97]:
from sklearn.metrics.pairwise import pairwise_distances

# User Similarity Matrix
user_correlation = 1 - pairwise_distances(train_data, metric='correlation')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation[:4, :4])

[[1.         0.74428238 0.99895282 0.99051878]
 [0.74428238 1.         0.77405931 0.64547613]
 [0.99895282 0.77405931 1.         0.9831962 ]
 [0.99051878 0.64547613 0.9831962  1.        ]]


In [98]:
# Item Similarity Matrix
item_correlation = 1 - pairwise_distances(train_data_matrix.T, metric='correlation')
item_correlation[np.isnan(item_correlation)] = 0
print(item_correlation[:4, :4])

[[ 1.          0.06982269 -0.10474043]
 [ 0.06982269  1.         -0.05820099]
 [-0.10474043 -0.05820099  1.        ]]


With the similarity matrix in hand, I can now predict the ratings that were not included with the data. Using these predictions, I can then compare them with the test data to attempt to validate the quality of our recommender model.

For the user-user CF case, I will look at the similarity between 2 users (A and B, for example) as weights that are multiplied by the ratings of a similar user B (corrected for the average rating of that user).

In [0]:
#TODO Function to predict ratings
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        rat_dif = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(rat_dif) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [0]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSE
def rmse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    print(pred)
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

In [106]:
# Predict ratings on the training data with both similarity score
user_prediction = predict(train_data_matrix, user_correlation, type='user')
item_prediction = predict(train_data_matrix, item_correlation, type='item')

# RMSE on the test data
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

[ 59.58086123  51.67159992   3.74753885  58.72940945  27.61384837
   6.65674218  54.87491324  45.66222755  -1.53714079  73.78339648
  69.60845792  19.6081456   39.22773082  44.29451233 -10.52224316
  66.36183456  41.74308892   9.89507652  22.61118177  52.99466423
  -1.605846    47.82793597  61.43654733   2.7355167   43.70503632
  10.49055786  -6.19559417  55.84411932  53.61189197   2.54398872
  15.89068105  48.60844683  -3.49912788  68.33249468  65.09432712
  14.5731782   60.72097613  50.4430692    3.83595468  63.2014341
  64.28466192  11.51390397  71.78316461  64.14612457  16.07071082
  34.98795098  40.53868359 -14.52663457  42.66491863  52.18458571
  -4.84950434  75.7996333   75.67550709  23.52485961  37.00957931
  63.40324257   5.58717812  45.76089657  31.02850431 -12.78940089
  76.32183533  77.83482708  24.84333759  62.18727339  41.96604834
   3.84667826  64.60660242  54.56631258   7.827085    66.33692286
  56.16438912   9.49868802  59.71569812  74.7062061   15.57809578
  64.012504